# Batch vs Mini-batch vs stochastic gradient descent

In the previous sections, neural networks models were trained on the entire training set containing m examples. This means that, in order to compte the cost function and update the model parameters, a complete pass, called *epoch* over all the $m$ training examples is inevitable. This may be extremely time consuming in this deep learning regime where the size of the training set is huge.

This motivates the idea to run the gradient descent on a subset of the $m$ training examples and update the parameters. These subsets are refered to as batches and the gradient descent is referred to as **mini-batch gradient descent**. It usually run faster than the gradient descent with all training examples, $m$.

In the case where the batches are of size 1, the gradient descent is called a **stochastic gradient descent**.

In the case where the batch size is $m$, the gradient descent algorithm is just a **batch gradient descent**.

## The implementation

Below is a pseudo-code implementation for the gradient descent with $T$ batches

```
repeat for t=1 to t=T:
  get the batch x_t
  calculate forward propagation with x_t
  compte cost for batch t
  apply back propagation
  update weights
```

When training batch vs mini-batch gradient descent, the cost function tends to descent smoothly in the case of batch gradient descent. However, in the case of mini-batch gradient descent, the general trend of the cost graph will go down as the number of iteration increases however the graph is oscillating on contrast to smooth behavior on the batch gradient descent. Generally, as the mini-batch size $t$ is closer to 1, the graph oscillation increases. In the extreme case of stochastic gradient descent, the algorithm may not reach to the global optimum due to the noisy oscillation it makes while descending. Consider the following image, taken from AndrewNg course, describing such situation.

![batch vs minibatch gradient descent](images/batch_vs_minibatch_vs_stochastic.png)

The image is a plot for the contours of the gradient descent while going down to the global minimum. The magenta oscillating graph represent the case of stochastic gradient descent while the blue smooth almost linear graph represents the batch gradient descent training. The green graph represents the case of mini-batch gradient descent with reasonable batch size $t$. However, reaching the global minimum is, also, not guaranteed.

## General guidelines on choosing the mini-batch size $t$

- If the training set is small, the batch gradient descent is the best candidate.
- If the training set is quite large, the batch size can be in between [64,512]
- Generally, setting $t$ to be a power of 2 performs better.
- Make sure the training mini-batch fits on the GPU/CPU memory.

# Exponentially weighted average

exponentially weighted average is a statistical tool to estimate the average of variable values from its history. For example, to estimate the average of day's temperature based on the previous days temperatures history.

The general formula of exponentially weighted average, sometimes called moving average, is:

$$V_{t} = \beta V_{t-1} + (1-\beta)\theta_{t}$$

Couple of notes on the parameter $\beta$:

- $\beta \in [0,1]$
- controls how the history affects the moving average. As long as $\beta$ increases, we are given more weights to the previous history. In particular, the number of values taken from the history is approximately $\frac{1}{1-\beta}$.
- for small values of $\beta$, the moving average becomes more susceptible to outliers. 
- The graph for small values or $\beta$ is more noisy and becomes smoother as $\beta$ increases.

## Bias correction

In the initial run of the moving average, it may not give the best estimate as $V_0=0$. However, setting $V_t = \frac{V_t}{1-\beta^2}$ gives better approximation

# Gradient Descent with Momentum

The idea behind momentum is to reduce the oscillations/noise resulting, for example,  from mini-batch algorithms.The idea is to average the gradients on each batch updates. The average algorithm used is the moving average discussed in the previous section.

## The implementation

on iteration t:
```
compute dw,db
Vdw = BVdw + (1-B)dw
Vdb = BVdb + (1-B)db
update w = w-alpha Vdw
update b = b- alpha Vdb
```

# RMSProp

RMSProb stands for Root Mean Square prob. This is another optimization to the gradient descent algorithm that tries to address the problem of noisy oscillation of the gradient descent while trying to climbing down towards the global minimum.

## How it works?

on iteration t:

```
compute dw,db on the current mini-batch
Sdw = B*Sdw + (1-B) dw2
Sdb = B*Sdb + (1-B) db2
w := w= a*(dw/sqrt{Sdw})
b := b= a*(db/sqrt{Sdb})
```

Notice that the main difference between the RMSProb and the momentum is that in the RMSProb formula, the derivatives are divided by the square root of the weighted average unlike the momentum where the parameters are just subtracted from their weighted average.

We hope that db is relatively large, as it is only one dimensional vector while dw is relatively small as it is high dimensional vector. Hence, the oscillations on the b dimensions got more smaller and smoother. The net effect of that the updates moves faster and smoother with a larger learning rate $\alpha$. In practice, the oscillations might be on the direction with not only db but many other parameters from dw. But the concept remains the same

# Adam algorithm

Adam optimization algorithm is nothing but a combination of RMSProb and Momentum. It stands for *ADAptive Moment estimation* Here is how it works.

```
Vdw=0, Sdw=0, Vdb=0, Sdb=0
on iteration t:
  compute dw,db using current mini-batch
  Vdw = B1*Vdw + (1-B1)dw, Vdb = B1*Vdb + (1-B1)db
  Sdw = B2*Sdw + (1-B2)dw^2, Sdb=B2*Sdb + (1-B2)db^2
  Vdw[corrected] = Vdw/(1-B1^t), Vdb[corrected] = Vdb/(1-B1^t)
  Sdw[corrected] = Sdw/(1-B1^t), Sdb[corrected] = Sdb/(1-B1^t)
  w = w-alpha*(Vdw[corrected]/(sqrt{Sdw[corrected]}+epsilon)), b=b-alpha*(Vdb[corrected]/(sqrt{Sdb[corrected]}+epsilon))
```

## common choices for the hyperparameters

- $\alpha$:needs to be tuned 
- $\beta_1$:0.9
- $\beta_2$:0.999
- $\epsilon:10^{-8}$